In [1]:
import numpy as np
import pandas as pd


# Line angles


In [2]:
from mechaphlowers.core.geometry.points import Points, Frame
from mechaphlowers.core.geometry.rotation import rotation_quaternion_same_axis, rotation_quaternion
from mechaphlowers.entities.arrays import SectionArray


In [3]:
section_array = SectionArray(
	pd.DataFrame(
		{
			"name": np.array(["support 1", "2", "three", "support 4"]),
			"suspension": np.array([False, True, True, False]),
			"conductor_attachment_altitude": np.array([30, 40, 60, 70]),
			"crossarm_length": np.array([-40, -40, -40, -40]),
			"line_angle": np.array([0, -45, -30, 60]),
			"insulator_length": np.array([0, 5, 10, 0]),
			"span_length": np.array([500, 500, 500, np.nan]),
		}
	)
)
section_array.sagging_parameter = 2000
section_array.sagging_temperature = 15

## First method: using projections to calculate coordinates

In [4]:
class LineAngleProjection:
	def __init__(self, section_array: SectionArray):
		self.section_array = section_array

	def project_x_right(
		self,
		crossarm_length: np.float64,
		line_angle: np.float64,
		dep_x: np.float64,
		dep_y: np.float64,
	):
		return (-crossarm_length + dep_y) * np.sin(line_angle / 2) + dep_x * np.cos(line_angle / 2)

	def project_x_left(
		self,
		crossarm_length: np.float64,
		line_angle: np.float64,
		dep_x: np.float64,
		dep_y: np.float64,
	):
		return (-crossarm_length + dep_y) * np.sin(-line_angle / 2) - dep_x * np.cos(-line_angle / 2)
		

	def project_y_right(
		self,
		crossarm_length: np.float64,
		line_angle: np.float64,
		dep_x: np.float64,
		dep_y: np.float64,
	):
		return (-crossarm_length + dep_y) * np.cos(line_angle / 2) + dep_x * np.sin(line_angle / 2)

	def project_y_left(
		self,
		crossarm_length: np.float64,
		line_angle: np.float64,
		dep_x: np.float64,
		dep_y: np.float64,
	):
		return (-crossarm_length + dep_y) * np.cos(-line_angle / 2) - dep_x * np.sin(-line_angle / 2)
		
	

	def get_new_span_lengths(
		self,
	):
		span_length = self.section_array.data.span_length.to_numpy()
		crossarm_length = self.section_array.data.crossarm_length.to_numpy()
		line_angle = self.section_array.data.line_angle.to_numpy()
		line_angle_radians = np.radians(line_angle)
		new_span_lengths = []
		lenghts_left = []
		lengths_right = []
		for index in range(len(span_length) - 1):
			current_span_length = span_length[index]
			current_crossarm_length = crossarm_length[index]
			current_line_angle = line_angle_radians[index]
			next_crossarm_length = crossarm_length[index + 1]
			next_line_angle = line_angle_radians[index + 1]
			# coords of the first support
			x_left = self.project_x_left(current_crossarm_length, current_line_angle, 0, 0)
			y_left = self.project_y_left(current_crossarm_length, current_line_angle, 0, 0)
			# coords of the second support
			x_right = self.project_x_right(next_crossarm_length, next_line_angle, 0, 0)
			y_right = self.project_y_right(next_crossarm_length, next_line_angle, 0, 0)
			lenghts_left.append(np.array([x_left, y_left]))
			lengths_right.append(np.array([x_right, y_right]))
			x_length = current_span_length - (x_right - x_left)
			# sign may be wrong but doesn't matter here
			y_length = y_right - y_left
			a_chain = (x_length ** 2 + y_length ** 2) ** 0.5
			new_span_lengths.append(a_chain)
		new_span_lengths.append(np.nan)
		print("lengths left:\n", np.array(lenghts_left))
		print("lengths right:\n", np.array(lengths_right))
		return np.array(new_span_lengths)
	


In [5]:
projection_model = LineAngleProjection(section_array)
projection_model.get_new_span_lengths()

lengths left:
 [[-0.         40.        ]
 [15.30733729 36.9551813 ]
 [10.3527618  38.63703305]]
lengths right:
 [[-15.30733729  36.9551813 ]
 [-10.3527618   38.63703305]
 [ 20.          34.64101615]]


array([515.31633274, 525.66278964, 490.36904384,          nan])

## Second method: using rotation module to avoid using projections

In [15]:

class LineAngleRotationQuaternion:
	def __init__(self, section_array: SectionArray):
		self.section_array = section_array
		self.init_frames()

	def init_frames(self):
		span_length = section_array.data.span_length.to_numpy()
		line_angle = self.section_array.data.line_angle.to_numpy()
		coords_origin = np.zeros((span_length.size, 3))
		frame = Frame(coords_origin)
		nb_supports = len(span_length)
		for support_index in range(nb_supports - 1):
			translation_current_span = np.array([[span_length[support_index], 0, 0]])
			translation_needed = np.repeat(translation_current_span, nb_supports-(support_index + 1), axis=0)
			translation_vector = np.concat((np.zeros((support_index + 1, 3)), translation_needed))
			frame.translate_current(translation_vector)
			rotation_needed = np.repeat(np.array([line_angle[support_index + 1]]), nb_supports-(support_index + 1))
			rotation_angles =  np.concat((np.zeros((support_index + 1,)), rotation_needed))
			frame.rotate_same_axis(rotation_angles, np.array([0, 0, 1]))

		self.frame = frame

	# TODO
	def attachment_coords_left(
		self,
	) -> Points:
		crossarm_length = self.section_array.data.crossarm_length.to_numpy()
		line_angle = self.section_array.data.line_angle.to_numpy()
		conductor_attachment_altitude = self.section_array.data.conductor_attachment_altitude.to_numpy()
		# create point before rotation: point at the end of the chain. Coords considering arm + chain
		# need to create frames properly:
		# best way: use relative transformations from previous frame 

		coords_points_arm = np.zeros((crossarm_length.size, 1, 3))
		points_left = Points(coords_points_arm, self.frame)
		# use translation method instead
		points_left.coords[:,:,1] = -crossarm_length[:, np.newaxis]
		points_left.coords[:,:,2] = conductor_attachment_altitude[:, np.newaxis]

	
		# [[0, crossarm_length_0, altitude_0],
		#  [0, crossarm_length_1, altitude_1],
		# ...
		# ]
		rotation_axis = np.array([0,0,1])
		points_left.rotate_same_axis(-line_angle/2, rotation_axis)
		points_attachment = self.coord_with_suspension_insulators(points_left)
		return points_attachment
	

	def attachment_coords_right(
		self,
	):
		# this should return the coords of the current support but considered in the plane of the previous one
		span_length = self.section_array.data.span_length.to_numpy()
		rolled_span_length = np.roll(span_length,1)
		crossarm_length = self.section_array.data.crossarm_length.to_numpy()
		line_angle = self.section_array.data.line_angle.to_numpy()
		conductor_attachment_altitude = self.section_array.data.conductor_attachment_altitude.to_numpy()
		coords_before_rot = np.zeros((crossarm_length.size, 3))
		coords_before_rot[:,1] = -crossarm_length
		coords_before_rot[:,2] = conductor_attachment_altitude
		# [[span_length_0, crossarm_length_0, altitude_0], 
		#  [span_length_1, crossarm_length_1, altitude_1],
		# ...
		# ]
		rotation_axis = np.array([0,0,1])


		coords_rotated = rotation_quaternion_same_axis(coords_before_rot, line_angle/2, rotation_axis)
		coord_attachment = self.coord_with_suspension_insulators(coords_rotated)


		coord_attachment[:,0] += rolled_span_length

		return coord_attachment
	

	def coord_with_suspension_insulators(
		self,
		points_end_of_crossarm: Points,
	) -> Points:
		# we assure suspension chains are vertical for now
		# use translation instead
		points_end_of_crossarm.coords[:,2] -= self.section_array.data.insulator_length.to_numpy()
		return points_end_of_crossarm


	def new_span_length(self):
		coords_supports_left = self.attachment_coords_left()
		coords_supports_right = self.attachment_coords_right()
		new_span_lengths = []
		for index in range(len(coords_supports_left) - 1):
			x_left, y_left = coords_supports_left[index][0:2]
			x_right, y_right = coords_supports_right[index  + 1][0:2]
			span_length = ((x_right - x_left) ** 2 + (y_right - y_left) ** 2) ** 0.5
			new_span_lengths.append(span_length)
		new_span_lengths.append(np.nan)
		return np.array(new_span_lengths)



In [16]:
rotation_model = LineAngleRotationQuaternion(section_array)
rotation_model.frame.origin


/home/wangjer/Projects/mechaphlowers/src/mechaphlowers/core/geometry/points.py:60: RuntimeWarning: invalid value encountered in divide
  rotation_axis_normalized = np.where(is_zero, default_z_axis, rotation_axis_reshaped/np.linalg.norm(rotation_axis_reshaped, axis=1)[:,np.newaxis])


array([[  0.        ,   0.        ,   0.        ],
       [500.        ,   0.        ,   0.        ],
       [853.55339059, 353.55339059,   0.        ],
       [982.96291314, 836.51630374,   0.        ]])

In [14]:
rotation_model.attachment_coords_left()

IndexError: index 2 is out of bounds for axis 1 with size 1

In [ ]:
rotation_model.attachment_coords_right()

array([[         nan,  40.        ,  30.        ],
       [515.30733729,  36.9551813 ,  35.        ],
       [510.3527618 ,  38.63703305,  50.        ],
       [480.        ,  34.64101615,  70.        ]])

In [ ]:
rotation_model.new_span_length()

array([515.31633274, 525.66278964, 490.36904384,          nan])